# developing the heat equation into a numeric approximation

$$\frac{\partial T}{\partial t}=k\frac{\partial^{2}T}{\partial x^{2}}$$

$$\lim_{dt\rightarrow0}\frac{T\left(t+dt,x\right)-T\left(t,x\right)}{dt}=k\lim_{dx\rightarrow0}\frac{T\left(x+dx,t\right)-2T\left(x,t\right),T\left(x-dx,t\right)}{dx}$$

## we will approximate
$$dt\approx0, dx\approx0$$

$$\frac{T\left(t+dt,x\right)-T\left(t,x\right)}{dt}\approx k\frac{T\left(t,x+dx\right)-2T\left(t,x\right)+T\left(t,x-dx\right)}{dx^{2}}$$

$$T\left(t+dt,x\right)\approx k\frac{\left(T\left(t,x+dx\right)-2T\left(t,x\right)+T\left(t,x-dx\right)\right)dt}{dx^{2}}+T\left(t,x\right)$$

$$T\left(t+dt,x\right)\approx k\frac{\left(\left(T\left(t,x+dx\right)-T\left(t,x\right)\right)-\left(T\left(t,x\right)-T\left(t,x-dx\right)\right)\right)dt}{dx^{2}}+T\left(t,x\right)$$

## Basic Functions
The following code contains the basic functions that will be used to simulate the heat equation.

In [64]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import plotly.graph_objects as go


length = 10
dx=0.1
k = 0.466
t = 10000
delta_t = 0.001


def heat_equation(simulation:np.ndarray,t:int,x:int,k:float,delta_t:float)->float:
    """ 
    function that accepts the location of a point in the rod and the time and returns it's temperature
    dT/dt = k/2((T3-T2)-(T2-T1)) is the change in temperature
    and then according to euler
    T = dT/dt*delta_t+T2)
    """
    T1=simulation[t,x-1]
    T2=simulation[t,x]
    T3=simulation[t,x+1]
    derTemp = k*(((T3-T2)-(T2-T1))/dx**2) 
    return derTemp*delta_t+T2

def heat_simulation(init_conditions:np.ndarray,p_amnt:float,k:float,t:float,delta_t:float):
    """ this constructs the 1d heat simulation
    """
    simulation = np.zeros(shape=(t,p_amnt+2))
    simulation[0,1:p_amnt+1]=init_conditions
    for i in range(1,t):
        for j in range(1,p_amnt+1):
            simulation[i,j]=heat_equation(simulation,i-1,j,k,delta_t)
        simulation[i,p_amnt+1]=simulation[i,p_amnt-1]
        simulation[i,0]=simulation[i,2]
    return simulation

## Simulation

In [65]:
# Define grid
x_values = np.linspace(0, length, int(length / dx) + 2)
t_values = np.arange(0, t, 1)
X, T = np.meshgrid(x_values, t_values)

init_conditions = np.sin(2*np.pi / 10 * x_values)
simulation = heat_simulation(init_conditions,int(length/dx)+2,k,t,delta_t)
simulation

array([[ 0.00000000e+00,  0.00000000e+00,  6.21696374e-02, ...,
        -6.21696374e-02, -2.44929360e-16,  0.00000000e+00],
       [ 6.21584291e-02,  2.89710510e-03,  6.21584291e-02, ...,
        -6.21584291e-02, -2.89710510e-03, -6.21584291e-02],
       [ 6.22822279e-02,  8.42026050e-03,  6.22822279e-02, ...,
        -6.22822279e-02, -8.42026050e-03, -6.22822279e-02],
       ...,
       [ 5.31626286e-01,  5.31849355e-01,  5.31626286e-01, ...,
        -5.31626286e-01, -5.31849355e-01, -5.31626286e-01],
       [ 5.31605493e-01,  5.31828565e-01,  5.31605493e-01, ...,
        -5.31605493e-01, -5.31828565e-01, -5.31605493e-01],
       [ 5.31584698e-01,  5.31807774e-01,  5.31584698e-01, ...,
        -5.31584698e-01, -5.31807774e-01, -5.31584698e-01]])

## 3D Plotting of the Simulation
Here we plot the simulation in 3D using plotly. The axes are the position on the rod, the time, and the temperature.

In [66]:

# Create figure
fig = go.Figure(data=[go.Surface(x=X, y=T, z=simulation, colorscale='viridis')])

# Update layout
fig.update_layout(scene=dict(xaxis_title='Position',
                             yaxis_title='Time',
                             zaxis_title='Temperature'),
                  title='Temperature Distribution Over Time',
                  template='plotly_white')

# Save as interactive HTML file
fig.write_html('interactive_plot.html') 

## Animation of the Simulation
Here we animate the simulation using matplotlib. The x-axis is the position on the rod, and the y-axis is the temperature.

In [67]:
# # Function to update the data for each frame of the animation
# def update(frame):
#     fig.data[0].z = simulation[frame]
#     return fig
# 
# # Define initial data
# init_conditions = np.sin(np.linspace(0, length, int(length / dx) + 2))
# simulation = heat_simulation(init_conditions, int(length / dx) + 2, k, t, delta_t)
# 
# # Create the figure
# x_values = np.linspace(0, length, int(length / dx) + 2)
# fig = go.Figure(data=[go.Surface(z=simulation[0], colorscale='viridis')])
# 
# # Update layout
# fig.update_layout(scene=dict(xaxis_title='Position',
#                              yaxis_title='Time',
#                              zaxis_title='Temperature'),
#                   title='Temperature Distribution Over Time')
# 
# # Create the animation
# frames = [go.Frame(data=[go.Surface(z=simulation[frame])], name=str(frame)) for frame in range(t)]
# fig.frames = frames
# 
# # Save animation as HTML
# fig.write_html("temperature_animation_plotly.html")